# Galaxy Morphology EDA & Classification 

### The URL for where dataset was acquired can be found [here](https://www.inserturl.com).

In [1]:
from utils.mypytable import MyPyTable 

table = MyPyTable()
file_name = 'SaYoPillow.csv'
table.load_from_file('input_folder/' + file_name)

print(table.data[0:5])
print(table.column_names)


[[93.8, 25.68, 91.84, 16.6, 89.84, 99.6, 1.84, 74.2, 3.0], [91.64, 25.104, 91.552, 15.88, 89.552, 98.88, 1.552, 72.76, 3.0], [60.0, 20.0, 96.0, 10.0, 95.0, 85.0, 7.0, 60.0, 1.0], [85.76, 23.536, 90.768, 13.92, 88.768, 96.92, 0.768, 68.84, 3.0], [48.12, 17.248, 97.872, 6.496, 96.248, 72.48, 8.248, 53.12, 0.0]]
['\ufeffsr', 'rr', 't', 'lm', 'bo', 'rem', 'sr', 'hr', 'sl']


## Class Distribution

#### We are hoping to train our model on the class label for morphology classification which is a system used by astronomers to divide galaxies into groups based on their visual appearance.

#### In order to ensure that we can fit a good model to the data, having a distribution of class labels that are not too skewed so as to be overly abundant or lacking in some versus others 

#### From our initial EDA we see that our class distribution .......

## A Closer Look: Features of Interest

#### Like the distributions of class lables we also used the opportunity for EDA to look into whether or not there were any features within the dataset that have high or low correlations with other attributes/class labels so as to gain intuition for potential pruning strategies or attributes to eliminate to reduce overfitting

###

### Preliminary Classification

In [2]:
cols_to_drop = []
target_col = []